In [103]:
import pandas as pd
from cssselect import GenericTranslator
from lxml.etree import XPath
from lxml import html
from bs4 import BeautifulSoup
import wget
from zipfile import ZipFile
import fnmatch
import os

In [7]:
# User Agent
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
}

In [8]:
# Wikipedia site with social revolts records
url = "https://en.wikipedia.org/wiki/List_of_revolutions_and_rebellions"

In [9]:
respuesta = requests.get(url, headers=headers)

In [10]:
# HTLM tree parser

parser = html.fromstring(respuesta.text)


## This is working fine.
### Here I parse the text of a given decade block from the wikipedia page using *xpath*

In [ ]:
# ingles = parser.get_element_by_id('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
ingles = parser.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
for i in ingles:
    print (i.text_content())

## This is working fine too. A different method with css selectors
### Here I parse the text of a given decade block from the wikipedia page using *css selectors.*

In [49]:
year = parser.xpath(GenericTranslator().css_to_xpath("ul:nth-child(100) li"))
for text in year:
    print(text.text_content())

2000–2005: The Second Intifada, a continuation of the First Intifada, between Palestinians and Israel.
2000: The bloodless Bulldozer Revolution, first of the four colour revolutions (in 2000, 2003, 2004, and 2005), overthrows Slobodan Milošević's régime in Yugoslavia.
2001: The 2001 Macedonia conflict.
2001–present: The Taliban insurgency following the 2001 war in Afghanistan which overthrew Taliban rule.
2001: The 2001 EDSA Revolution peacefully ousts Philippine President Joseph Estrada after the collapse of his impeachment trial.
2001: Supporters of Philippines former president Joseph Estrada violently and unsuccessfully stage a rally, so-called the EDSA Tres, in an attempt of returning him to power.
2001: Cacerolazo in Argentina. Following mass riots and a period of civil unrest, popular protests oust the government and two additional interim presidents within months. December 2001 riots in Argentina
2003–2005: Bolivian gas conflict.
2003: The Rose Revolution, second of the colour r

## Finidng the links
### here you can find the links for a given decade

In [75]:
links = parser.xpath(GenericTranslator().css_to_xpath('ul:nth-child(100) a'))
for link in links:
#     print(link.text_content())
    print('https://en.wikipedia.org' + link.attrib['href'])

https://en.wikipedia.org/wiki/Second_Intifada
https://en.wikipedia.org/wiki/First_Intifada
https://en.wikipedia.org/wiki/Palestinian_people
https://en.wikipedia.org/wiki/Israel
https://en.wikipedia.org/wiki/Bulldozer_Revolution
https://en.wikipedia.org/wiki/Colour_revolutions
https://en.wikipedia.org/wiki/Slobodan_Milo%C5%A1evi%C4%87
https://en.wikipedia.org/wiki/Federal_Republic_of_Yugoslavia
https://en.wikipedia.org/wiki/2001_Macedonia_conflict
https://en.wikipedia.org/wiki/Taliban_insurgency
https://en.wikipedia.org/wiki/War_in_Afghanistan_(2001%E2%80%93present)
https://en.wikipedia.org/wiki/EDSA_Revolution_of_2001
https://en.wikipedia.org/wiki/List_of_Presidents_of_the_Philippines
https://en.wikipedia.org/wiki/Joseph_Estrada
https://en.wikipedia.org/wiki/Impeachment
https://en.wikipedia.org/wiki/List_of_Presidents_of_the_Philippines
https://en.wikipedia.org/wiki/Joseph_Estrada
https://en.wikipedia.org/wiki/EDSA_Tres
https://en.wikipedia.org/wiki/December_2001_riots_in_Argentina
htt

# Use another wikipedia page source 

## I look for a more simple page to scrap the dataset


In [78]:
url_2 = 'https://en.wikipedia.org/wiki/List_of_protests_in_the_21st_century'

In [80]:
respuesta_2 = requests.get(url_2, headers=headers)

In [81]:
# HTLM tree parser

parser_2 = html.fromstring(respuesta_2.text)


protest = parser_2.xpath(GenericTranslator().css_to_xpath('dt'))
for text in protest:
    print(text.text_content())
    print()

## Extract socioeconomical data from the worlbank

In [ ]:
WB_url_seed = 'http://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD'
my_params = {'date' : '2000:2020',
            'incomelevel' :'',
            'downloadformat' : 'csv',
            'per_page' : '304'} # dic with the parameters of interest

In [ ]:
response_socio_param = requests.get(WB_url_seed, params = my_params)

In [98]:
my_zip_file = wget.download(response_socio_param.url)

In [ ]:
with ZipFile(my_zip_file, 'r') as zipObj:
            for content in zipObj.namelist():
                if fnmatch.fnmatch(content, 'API_*'):
                    #print(content) #wihtin the content of the zip file find and extract the csv file that contain the data
                    my_filename = content
                    zipObj.extract(content)

In [ ]:
path = os.getcwd()

In [ ]:
path + "/" + my_filename

In [ ]:
my_df = pd.read_csv(path + '/' + my_filename, 
               header = 2,
                   )

In [10]:
# HTLM tree parser

parser = html.fromstring(respuesta.text)


In [119]:
my_df

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 25
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,1.873453e+09,1.920112e+09,1.941341e+09,2.021229e+09,2.228492e+09,2.330726e+09,...,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,4.055180e+09,4.515559e+09,5.226779e+09,6.209138e+09,...,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,NaN,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,9.129595e+09,8.936064e+09,1.528559e+10,1.781271e+10,2.355205e+10,3.697092e+10,...,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.881570e+10,NaN,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,3.480355e+09,3.922099e+09,4.348070e+09,5.611492e+09,7.184681e+09,8.052076e+09,...,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,NaN,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,1.429049e+09,1.546926e+09,1.755910e+09,2.361727e+09,2.894922e+09,3.159905e+09,...,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,GDP (current US$),NY.GDP.MKTP.CD,1.849196e+09,2.535334e+09,2.406271e+09,2.790456e+09,3.556757e+09,3.663102e+09,...,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,NaN,NaN
260,"Yemen, Rep.",YEM,GDP (current US$),NY.GDP.MKTP.CD,9.652436e+09,9.861560e+09,1.069463e+10,1.177797e+10,1.387279e+10,1.674634e+10,...,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,NaN,NaN
261,South Africa,ZAF,GDP (current US$),NY.GDP.MKTP.CD,1.363613e+11,1.215147e+11,1.154824e+11,1.752569e+11,2.285900e+11,2.577727e+11,...,3.963327e+11,3.668294e+11,3.509046e+11,3.176205e+11,2.963573e+11,3.495541e+11,3.682889e+11,3.514316e+11,NaN,NaN
262,Zambia,ZMB,GDP (current US$),NY.GDP.MKTP.CD,3.600683e+09,4.094481e+09,4.193846e+09,4.901840e+09,6.221078e+09,8.331870e+09,...,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,NaN,NaN
